3*28*28을 flatten해서 3*784로 바꿈
nn.Lenear에서는 W의 사이즈를 설정해줌
Z=X*W에서 W의 size 결정

ReLU
Z=X*W는 Linear, 하지만 딥러닝에서 혹은 머신러닝에서 완벽히 Linear한건 좋은 결과를 얻지 못함, 그래서 X*W에 ReLU함수를 통과시켜서 Non-linear화 시킴
Z=ReLU(X*W)
0보다 작은 값들을 0으로 바꿈, 0보다 큰건 그대로

sequential에선 위에서 정의한 함수들을 순서대로 실행함
ReLU함수는 parameter없음

In [2]:
#미분 과정
#Gradient descent에서 미분하는 방법
import torch

x = torch.ones(5)  # input tensor, 1*5
y = torch.zeros(3)  # expected output 1*3
w = torch.randn(5, 3, requires_grad=True)# 5*3
#b는 bias
b = torch.randn(3, requires_grad=True)#1*3
#z=x*w+b
#학습은 w랑 b만
z = torch.matmul(x, w)+b
#z는 예측, y는 정답, 따라서 z와 y를 비교해 loss를 구함
#두 개가 비슷하면 loss는 작고, 반대라면 loss 큼
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

따라서 loss 따라 미분해 w를 업데이트함

In [3]:
loss.backward()
#w와 b의 미분값을 이용해 업데이트 해야 함
print(w.grad)
print(b.grad)

tensor([[0.3166, 0.1394, 0.0581],
        [0.3166, 0.1394, 0.0581],
        [0.3166, 0.1394, 0.0581],
        [0.3166, 0.1394, 0.0581],
        [0.3166, 0.1394, 0.0581]])
tensor([0.3166, 0.1394, 0.0581])


In [4]:
z = torch.matmul(x, w)+b
#z가 미분이 필요한지 여부 확인
print(z.requires_grad)

#여기는 미분이 필요 없다고 함
#이런 함수를 통해 w를 업데이트 하지 않을 필요가 있을 때 사용
#test를 할때는 정답을 모르므로 예측만 하고 업데이트를 하지않음
with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
#detach함수는 z에서 떼네어 z가 텐서가 아니도록 뗴어냄
#따라서 z를 미분하지 않음
print(z_det.requires_grad)

False


In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [15]:
#파라미터는 w, b등 학습을 통해 업데이트 하는 것
#하이퍼파라미터는 사용자가 세팅해주는 값, 유저 인풋
#러닝 레이트는얼마나 빨리 학습할것인지
#하산을 할 때 기울기를 보고 하산하는데, 한번 내려오는데 얼마만큼 내려올 것인지 결정하는 것
#러닝 레이트가 높으면 기울기에 따라서 많이 학습함
#가장 효과적인 러닝 레이트? 경험적으로 사람이 직접 실습해보며 직관적으로 설정해야함
learning_rate = 1e-3
#한번에 얼마만큼의 데이터를 사용해 학습할건지
batch_size = 64
#트레이닝 데이터 전체를 학습한게 1, 만약 train data가 10000장이면 50000장을 학습하는 것
epochs = 5

In [16]:
# Initialize the loss function
#현재 딥러닝 모델이 얼마나 잘 학습되었는지 판단하는 척도
#학습이 잘 되어있으면 조금만, 학습이 잘 안되어있으면 많이 학습해야함
loss_fn = nn.CrossEntropyLoss()

In [17]:
#우리는 stochasic gradient descent 사용
#많은 optimizer중 하나
#optimizer로 실제로 학습함
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [18]:
#실제로 학습하는 과정
def train_loop(dataloader, model, loss_fn, optimizer):
    #train data 총 몇개인지 확인
    size = len(dataloader.dataset)
    #64개씩 가져옴
    #X는 데이터, y는 레이블
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        #zero grad는 optimizer 초기화, 현재 0 상태로 설정
        optimizer.zero_grad()
        #loss 미분해 w, b에 대한 미분값 계산
        loss.backward()
        #실제로 학습시키기
        optimizer.step()

        #현재 얼마만큼 학습되고 있는지 출력
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    #test에서는 업데이트 하지 않고 테스트만 함
    #학습 x
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            #정답의 개수 더함
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

#모델 저장
model.load_state_dict(torch.load('model_weights.pth'))

Epoch 1
-------------------------------
loss: 2.297317  [    0/60000]
loss: 2.295680  [ 6400/60000]
loss: 2.282171  [12800/60000]
loss: 2.278173  [19200/60000]
loss: 2.280745  [25600/60000]
loss: 2.271203  [32000/60000]
loss: 2.270566  [38400/60000]
loss: 2.258876  [44800/60000]
loss: 2.262221  [51200/60000]
loss: 2.226197  [57600/60000]
Test Error: 
 Accuracy: 34.7%, Avg loss: 0.034927 

Epoch 2
-------------------------------
loss: 2.251766  [    0/60000]
loss: 2.247536  [ 6400/60000]
loss: 2.203068  [12800/60000]
loss: 2.193379  [19200/60000]
loss: 2.188465  [25600/60000]
loss: 2.167852  [32000/60000]
loss: 2.189089  [38400/60000]
loss: 2.151186  [44800/60000]
loss: 2.184234  [51200/60000]
loss: 2.100629  [57600/60000]
Test Error: 
 Accuracy: 42.8%, Avg loss: 0.032968 

Epoch 3
-------------------------------
loss: 2.171686  [    0/60000]
loss: 2.152184  [ 6400/60000]
loss: 2.056853  [12800/60000]
loss: 2.047929  [19200/60000]
loss: 2.011780  [25600/60000]
loss: 2.005525  [32000/600

In [22]:
#어떤 알고리즘이나 TEST DATA 바꾸고 싶음
#매번 다시 학습하기 보다는 한번 학습한걸 저장해놓고 다시 사용할 수 있음
#W나 B를 저장해놓고 다시 쓰는 것
import torch
import torch.onnx as onnx
import torchvision.models as models

In [25]:
#PRETAIN모델 - 기존에 굉장히 큰 데이터셋으로 학습시킨 모델
#이걸 다시 학습할려면 굉장히 오래 걸림
#그 모델의 W와 B를 가져옴
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

In [26]:
#모델 만들고 그 WEIGHT를 불러오면 됨
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1